# Data Enrichment Project

# Imports & Data Cleaning


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, time, json, math
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
# defining function for movie rating
# CITING: Using TMDB API from LP
"""Adapated from https://github.com/celiao/tmdbsimple"""

def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
            
    return info

In [3]:
# defining function for writing json
# CITING: Efficient TMDB API calls from LP
"""Adapted from https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

def write_json(new_data, filename):
    
    with open(filename,'r+') as file:
        file_data = json.load(file)
        
        if(type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
            
        file.seek(0)
        json.dump(file_data,file)

In [4]:
# creating fucntion for json file error
# CITING: 
"""source: https://github.com/coding-dojo-data-science/data-enrichment-helper-functions"""
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [5]:
basic_gz = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_gz = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_gz = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [6]:
basics = pd.read_csv(basic_gz, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_gz, sep='\t', low_memory=False)
akas = pd.read_csv(akas_gz, sep='\t', low_memory=False)

**Basics DF Data Cleanse**

In [7]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9329527 entries, 0 to 9329526
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 640.6+ MB


In [9]:
# replacing null values with np.nan
basics = basics.replace({'\\N':np.nan})
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [10]:
# eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['runtimeMinutes'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2545343 entries, 0 to 9329526
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 194.2+ MB


In [11]:
# eliminate movies that are null for genres
basics = basics.dropna(subset=['genres'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2477362 entries, 0 to 9329526
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 189.0+ MB


In [12]:
# keep only titleType==Movie
basics['titleType'].value_counts()

tvEpisode       1130029
short            580932
movie            370803
video            176259
tvMovie           88835
tvSeries          87519
tvSpecial         16890
tvMiniSeries      16348
tvShort            9437
videoGame           310
Name: titleType, dtype: int64

In [13]:
basics = basics[basics.titleType == 'movie']
basics['titleType'].value_counts()

movie    370803
Name: titleType, dtype: int64

In [14]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
930,tt0000941,movie,Locura de amor,Locura de amor,0,1909,NaN,45,Drama


In [15]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370803 entries, 8 to 9329477
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          370803 non-null  object
 1   titleType       370803 non-null  object
 2   primaryTitle    370803 non-null  object
 3   originalTitle   370803 non-null  object
 4   isAdult         370803 non-null  object
 5   startYear       364877 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  370803 non-null  object
 8   genres          370803 non-null  object
dtypes: object(9)
memory usage: 28.3+ MB


In [16]:
# keep startYear 2000-2022
basics = basics.dropna(subset=['startYear'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364877 entries, 8 to 9329477
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          364877 non-null  object
 1   titleType       364877 non-null  object
 2   primaryTitle    364877 non-null  object
 3   originalTitle   364877 non-null  object
 4   isAdult         364877 non-null  object
 5   startYear       364877 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  364877 non-null  object
 8   genres          364877 non-null  object
dtypes: object(9)
memory usage: 27.8+ MB


In [17]:
basics['startYear'] = basics['startYear'].astype(int)
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364877 entries, 8 to 9329477
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          364877 non-null  object
 1   titleType       364877 non-null  object
 2   primaryTitle    364877 non-null  object
 3   originalTitle   364877 non-null  object
 4   isAdult         364877 non-null  object
 5   startYear       364877 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  364877 non-null  object
 8   genres          364877 non-null  object
dtypes: int32(1), object(8)
memory usage: 26.4+ MB


In [18]:
basics = basics.drop(basics[basics['startYear']<2000].index)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
66310,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,NaN,47,Documentary
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


In [19]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218631 entries, 13079 to 9329477
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          218631 non-null  object
 1   titleType       218631 non-null  object
 2   primaryTitle    218631 non-null  object
 3   originalTitle   218631 non-null  object
 4   isAdult         218631 non-null  object
 5   startYear       218631 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  218631 non-null  object
 8   genres          218631 non-null  object
dtypes: int32(1), object(8)
memory usage: 15.8+ MB


In [20]:
# Eliminate movies that include "Documentary" in genre
documentary_pos = basics['genres'].str.contains('documentary',case=False)
basics = basics[~documentary_pos]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77935,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144761 entries, 34793 to 9329376
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          144761 non-null  object
 1   titleType       144761 non-null  object
 2   primaryTitle    144761 non-null  object
 3   originalTitle   144761 non-null  object
 4   isAdult         144761 non-null  object
 5   startYear       144761 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  144761 non-null  object
 8   genres          144761 non-null  object
dtypes: int32(1), object(8)
memory usage: 10.5+ MB


In [22]:
# Keep only US movies
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34793      True
61095      True
67641      True
77935      True
86771      True
           ... 
9329199    True
9329208    True
9329247    True
9329292    True
9329376    True
Name: tconst, Length: 144761, dtype: bool

In [23]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77935,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [24]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144109 entries, 34793 to 9329376
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          144109 non-null  object
 1   titleType       144109 non-null  object
 2   primaryTitle    144109 non-null  object
 3   originalTitle   144109 non-null  object
 4   isAdult         144109 non-null  object
 5   startYear       144109 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  144109 non-null  object
 8   genres          144109 non-null  object
dtypes: int32(1), object(8)
memory usage: 10.4+ MB


**AKA's DF Data Cleanse**

In [25]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [26]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33647437 entries, 0 to 33647436
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


In [27]:
# Replace "\N" with np.nan
akas = akas.replace({'\\N':np.nan})
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [28]:
# keep only US movies
akas['region'].value_counts()

JP    4016768
FR    4015532
DE    4012992
IN    3947728
ES    3938498
       ...   
CC          1
TV          1
NU          1
PW          1
NR          1
Name: region, Length: 247, dtype: int64

In [29]:
akas = akas.dropna(subset=['region'])
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31780218 entries, 0 to 33647435
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [30]:
is_us = akas['region'].str.contains('US',case=False)
akas = akas[is_us]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [31]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1362871 entries, 5 to 33647181
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1362871 non-null  object
 1   ordering         1362871 non-null  int64 
 2   title            1362871 non-null  object
 3   region           1362871 non-null  object
 4   language         3704 non-null     object
 5   types            965305 non-null   object
 6   attributes       45243 non-null    object
 7   isOriginalTitle  1361496 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.6+ MB


**Ratings DF Data Clease**

In [32]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
2,tt0000003,6.5,1728
3,tt0000004,5.6,173
4,tt0000005,6.2,2541


In [33]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1241487 entries, 0 to 1241486
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1241487 non-null  object 
 1   averageRating  1241487 non-null  float64
 2   numVotes       1241487 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.4+ MB


In [34]:
# Keep only US movies
keepers1 = ratings['tconst'].isin(akas['titleId'])
keepers1

0           True
1           True
2          False
3          False
4           True
           ...  
1241482    False
1241483     True
1241484    False
1241485    False
1241486    False
Name: tconst, Length: 1241487, dtype: bool

In [35]:
ratings = ratings[keepers1]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
4,tt0000005,6.2,2541
5,tt0000006,5.1,175
6,tt0000007,5.4,797


In [36]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472717 entries, 0 to 1241483
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         472717 non-null  object 
 1   averageRating  472717 non-null  float64
 2   numVotes       472717 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


**Saving csv file**

In [37]:
basics.to_csv('Data/title.basics.csv.gz',compression='gzip',index=False)

In [38]:
basics = pd.read_csv('Data/title.basics.csv.gz',low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


**Loading API Credentials**

In [39]:
with open('/Users/chris/.secret/tmdb_api.json','r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [40]:
tmdb.API_KEY = login['api-key']

**Designating folder for API data**

In [41]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title.akas.tsv.gz',
 'title.basics.csv.gz',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

**Loading in dataframe**

In [42]:
basics = pd.read_csv('Data/title.basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


**YEAR 2000**

In [43]:
# defining lists of years to extract & errors list
YEARS_TO_GET = [2000]
errors = []

In [44]:
# starting outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='Years', position=0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    df = basics.loc[ basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
# starting inner loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                              desc=f'Movies from {YEAR}',
                              position=1,
                              leave=True):
    try:
        temp = get_movie_with_rating(movie_id)
        write_json(temp,JSON_FILE)
        time.sleep(0.02)
        
    except Exception as e:    
        errors.append([movie_id,e])
        
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz',compression='gzip',
                    index=False)

Years:   0%|          | 0/1 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/525 [00:00<?, ?it/s]

In [45]:
print(f'- Total errors: {len(errors)}')

- Total errors: 525


**YEAR 2001**

In [46]:
YEARS_TO_GET1 = [2001]
errors1 = []

In [47]:
for YEAR1 in tqdm_notebook(YEARS_TO_GET1, desc='Years', position=0):
    
    JSON_FILE1 = f'{FOLDER}tmdb_api_results_{YEAR1}.json'
    
    file_exists = os.path.isfile(JSON_FILE1)
    
    if file_exists == False:
        with open(JSON_FILE1,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    df1 = basics.loc[ basics['startYear']==YEAR1].copy()
    movie_ids1 = df1['tconst'].copy()
    previous_df1 = pd.read_json(JSON_FILE1)
    movie_ids_to_get1 = movie_ids1[~movie_ids1.isin(previous_df1['imdb_id'])]
    
for movie_id1 in tqdm_notebook(movie_ids_to_get1,
                               desc=f'Movies from {YEAR1}',
                               position=1,
                               leave=True):
    try:
        temp1 = get_movie_with_rating(movie_id1)
        write_json(temp1,JSON_FILE1)
        time.sleep(0.02)
    
    except Exception as e:
        errors1.append([movie_id1,e])
        
final_year_df1 = pd.read_json(JSON_FILE1)
final_year_df1.to_csv(f'{FOLDER}final_tmdb_data_{YEAR1}.csv.gz',compression='gzip',
                     index=False)

Years:   0%|          | 0/1 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/583 [00:00<?, ?it/s]

In [48]:
print(f'- Total errors: {len(errors1)}')

- Total errors: 583


In [49]:
test_ids = ['tt0848228','tt0332280']
results = []
errors = []

for id_movie in test_ids:
    
    try:
        movie_info = get_movie_with_rating(id_movie)
        results.append(movie_info)
        
    except Exception as e:
        errors.append([id_movie,e])

pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.706,27683,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,9880,PG-13


In [50]:
print(f'- Number of errors: {len(errors)}')
errors

- Number of errors: 0


[]